In [2]:
install.packages("sparklyr")
library(dplyr)
library(tidyverse)
library(ggplot2)
library(sparklyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘config’, ‘globals’



Attaching package: ‘sparklyr’


The following object is masked from ‘package:purrr’:

    invoke


The following object is masked from ‘package:stats’:

    filter




In [ ]:
#Creo conexión a Spark
sc=spark_connect(master="local")

#Cargo el archivo CSV a un RDD
rdd=spark_read_csv(sc,"Belice4_S1.csv",delimiter = "\t")

#Aplico transformaciones con dplyr
rdd_transformed= rdd %>% mutate(franja_horaria=substr(Time,1,2))

In [ ]:
#--------------------------------------------------------------------
#Si se quiere aplicar transformaciones con SQL (quizá sea mejor subir el
#CSV a Postgres y trabajarlo desde ahí con SQL directamente)
#--------------------------------------------------------------------

#Creo conexión a Spark
sc=spark_connect(master="local")

#Cargo el archivo CSV a un RDD
rdd=spark_read_csv(sc,"Belice4_S1.csv",delimiter = "\t")

#Creo sesión de Spark SQL
sqlContext=spark_session(sc)

#Registro el RDD como tabla temporal
table <- copy_to(sc, rdd, "mi_tabla")

#Aplico transformación con SQL
transformed_rdd=sqlContext %>%
  sql("SELECT * FROM mi_tabla WHERE ...")

In [ ]:
ds=read.csv("Belice4_S1.csv",sep='\t')
muestra= ds %>% sample_n(100000)

head(ds,10)
View(ds)

#Análisis

summary(muestra)
spec(muestra)

#creo variable franja horaria
muestra = muestra %>% mutate(franja_horaria=substr(Time,1,2))

muestra %>% summarise(max=max(Depth,na.rm = T),media=mean(Depth,na.rm = T))
muestra %>% group_by(Date,franja_horaria) %>% summarise(max=ifelse(max(Depth,na.rm = T)==-Inf,NA,max(Depth,na.rm = T)),media=mean(Depth,na.rm = T))
depth_X_hora=muestra %>% group_by(franja_horaria) %>% summarise(max=ifelse(max(Depth,na.rm = T)==-Inf,NA,max(Depth,na.rm = T)),media=mean(Depth,na.rm = T))

depth_X_hora %>% ggplot(aes(franja_horaria,media))+geom_line()
